# Zadanie 6

Celem ćwiczenia jest implementacja algorytmu Q-learning.

Następnie należy stworzyć agenta rozwiązującego problem [Taxi](https://gymnasium.farama.org/environments/toy_text/taxi/). Problem dostępny jest w pakiecie **gym**.

Punktacja (max 8 pkt):
- Implementacja algorytmu Q-learning. [3 pkt]
- Eksperymenty dla różnych wartości hiperparametrów [2 pkt]
- Jakość kodu [1.5 pkt]
- Wnioski [1.5 pkt]


In [15]:
import numpy as np
import gym
import random
import pygame
from IPython.display import clear_output

random.seed(1234)

In [14]:
# Interfejs

class QLearningSolver:
    """Class containing the Q-learning algorithm that might be used for different discrete environments."""
    def __init__(self,
                 observation_space:int,
                 action_space:int,
                 learning_rate:float=0.1,
                 gamma:float=0.9,
                 epsilon:float=0.1,
                 ):
        self.observation_space = observation_space
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.gamma = gamma # wsp znizki nagrody
        self.epsilon = epsilon # eksploracja
        self.q_table = np.zeros((observation_space, action_space))
        self.steps = []

    def __call__(self, state:np.ndarray, action:np.ndarray) -> np.ndarray:
        """Return Q-value of given state and action."""
        return self.q_table[state, action]

    def update(self, state:np.ndarray, action:np.ndarray, next_state:np.ndarray, reward:float) -> None:
        # state jako aktualny stan agenta, akcja jako czynnosc wykonywana przez agenta
        """Update Q-value of given state and action."""
        val = reward + self.gamma * np.max(self.q_table[next_state])
        act = (1-self.learning_rate) * self.q_table[state, action]
        self.q_table[state, action] = act + self.learning_rate * val

    def get_best_action(self, state:np.ndarray):
        """Return action that maximizes Q-value for a given state."""
        return np.argmax(self.q_table[state])

    def __repr__(self):
        """Elegant representation of Q-learning solver."""
        return("A")

    def __str__(self):
        return self.__repr__()

    def learn(self, episodes:int, iter_per_episode:int, env):
        for episode in(range(episodes)):
            state = env.reset()[0]
            done = False
            episode_reward = 0
            while not done:
                if(np.random.uniform(0, 1) < self.epsilon):
                    action = env.action_space.sample()
                else:
                    action = self.get_best_action(state)
                
                next_state, reward, done, info, a = env.step(action)

                self.update(state, action, next_state, reward)
                episode_reward += 1
                
                state = next_state

            self.steps.append(episode_reward)

            self.epsilon = max(0.01, np.exp(-0.001*episode))

In [10]:
# Find best learning rate


learning_rate = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
gamma:float=0.4 #discount rate
epsilon:float=0.1 #exploration rate
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

for nn in range(9):
    qlearn = QLearningSolver(streets.observation_space.n, streets.action_space.n, learning_rate[nn], gamma, epsilon)
    qlearn.learn(10000, 25, streets)
    print(np.mean(qlearn.steps[9000:10000]))

16.038
13.382
13.289
13.155
13.229
13.331
13.274
13.173
13.133


In [11]:
# Find best discount rate


learning_rate = 0.1
gamma:float=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] #discount rate
epsilon:float=0.1 #exploration rate
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

for nn in range(9):
    qlearn = QLearningSolver(streets.observation_space.n, streets.action_space.n, learning_rate, gamma[nn], epsilon)
    qlearn.learn(10000, 25, streets)
    print(np.mean(qlearn.steps[9000:10000]))

23.655
21.079
18.112
16.053
14.898
13.998
13.548
13.292
13.124


In [12]:
# Find best exp rate


learning_rate = 0.1
gamma:float = 0.1 #discount rate
epsilon:float=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] #exploration rate
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

for nn in range(9):
    qlearn = QLearningSolver(streets.observation_space.n, streets.action_space.n, learning_rate, gamma, epsilon[nn])
    qlearn.learn(10000, 25, streets)
    print(np.mean(qlearn.steps[9000:10000]))

23.485
23.477
23.757
23.926
23.572
23.601
23.559
23.66
23.656


In [17]:
learning_rate:float=0.5
gamma:float=0.9 #discount rate
epsilon:float=0.1 #exploration rate
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

qlearn = QLearningSolver(streets.observation_space.n, streets.action_space.n, learning_rate, gamma, epsilon)
qlearn.learn(10000, 25, streets)
for i in range(10):
    print(np.mean(qlearn.steps[1000*i:1000*(i+1)]))

c:\Users\niewi\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


168.019
17.842
14.512
13.576
13.315
13.207
13.245
13.178
13.114
13.104


# Eksperymenty

# Wnioski

1. Dzięki przeprowadzaniu uczenia się, algorytm jest w stanie znaleźć rozwiązanie problemu taxi, widać to po uśrednionej ilości kroków do osiągięcia celu,
w początkowej fazie liczba ta oscyluje w granicach 25 co oznacza że algorytm nie znajduje rozwiązania zadania, następnie sukcesywnie się zmniejsza i osiąga wartość około 13.
2. Optymalna ilość epok wynosi około 10.000, zwiększenie tej wartości do 30.000 nie poprawia działania alg., wartość średnia ilości kroków nie zmniejsza się w sposób znaczny
3. Dobór poszczególnych współczynników jest kluczowy dla poprawnego działania algorytmu, z przeprowadzonych ekperymentów wynika iż optymalne wartości to:
learning rate = 0.3
gamma = 0.9
epsilon = 0.7